In [1]:
from datetime import datetime
import plotly.express as px
import polars as pl
import os
import glob

DATA_DIR = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS-edge/ThingsBoard-Downloader/data"

start_date = datetime(2025, 2, 1, 0, 0, 0)
end_date = datetime(2025, 2 ,28, 23, 59, 59)

filter = '1h'
sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
sensor_id = [3,6]

In [2]:
data_paths = glob.glob(os.path.join(DATA_DIR, "2025", "*.parquet"))

dfs = []

for path in data_paths:
    dfs.append(pl.read_parquet(path).with_columns(system_name = pl.lit(os.path.basename(path).split(".")[0])))
    
df = pl.concat(dfs, how="diagonal") \
    .filter(pl.col("datetime").is_between(start_date, end_date)) \
    .with_columns(pl.col("system_name").str.extract(r'(\d+)',1).str.to_integer().alias("system_id"))
    

df = df.filter(pl.col("system_id").is_in(sensor_id))

In [3]:
column_name = "gmp343_edge_corrected"

df_plot = df.select(["datetime", "system_id", column_name]) \
      .group_by_dynamic("datetime", every=filter, group_by="system_id") \
      .agg(pl.col(column_name).mean())


fig = px.line(
    df_plot,
    x="datetime",
    y=column_name,
    markers=True,
    color="system_id"
)
fig.show()